### requirements for the following codings


In [ ]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [ ]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# training dataset loading
dataset = pd.read_excel('QS_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('QS_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [ ]:
# test dataset loading
dataset = pd.read_excel('QS_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('QS_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [ ]:
# assign the dataset 
X_train_data_name = 'QS_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'QS_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [ ]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(400, 320)
(40, 320)
(400,)
(40,)


### Model architecture

In [ ]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [ ]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


45/45 [==============================] - 2s 14ms/step - loss: 1.9002 - accuracy: 0.7972 - val_loss: 0.6819 - val_accuracy: 0.7250 - lr: 0.1000
Epoch 2/200
45/45 [==============================] - 1s 23ms/step - loss: 0.2500 - accuracy: 0.9167 - val_loss: 0.6367 - val_accuracy: 0.6250 - lr: 0.1000
Epoch 3/200
45/45 [==============================] - 1s 24ms/step - loss: 0.1890 - accuracy: 0.9333 - val_loss: 0.6521 - val_accuracy: 0.7500 - lr: 0.0600
Epoch 4/200
45/45 [==============================] - 1s 21ms/step - loss: 0.1466 - accuracy: 0.9444 - val_loss: 0.6033 - val_accuracy: 0.7500 - lr: 0.0600
Epoch 5/200
45/45 [==============================] - 1s 14ms/step - loss: 0.1204 - accuracy: 0.9611 - val_loss: 0.5635 - val_accuracy: 0.9000 - lr: 0.0600
Epoch 6/200
45/45 [==============================] - 1s 18ms/step - loss: 0.0855 - accuracy: 0.9750 - val_loss: 0.5069 - val_accuracy: 0.9250 - lr: 0.0360
Epoch 7/200
45/45 [==============================] - 1s 19ms/step - loss: 0.0830 -

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


45/45 [==============================] - 1s 12ms/step - loss: 1.9529 - accuracy: 0.7361 - val_loss: 0.5824 - val_accuracy: 0.8250 - lr: 0.1000
Epoch 2/200
45/45 [==============================] - 0s 10ms/step - loss: 0.3013 - accuracy: 0.8917 - val_loss: 0.6658 - val_accuracy: 0.5500 - lr: 0.1000
Epoch 3/200
45/45 [==============================] - 0s 11ms/step - loss: 0.2497 - accuracy: 0.9083 - val_loss: 0.6475 - val_accuracy: 0.5500 - lr: 0.0600
Epoch 4/200
45/45 [==============================] - 0s 10ms/step - loss: 0.1896 - accuracy: 0.9278 - val_loss: 0.6363 - val_accuracy: 0.6000 - lr: 0.0600
Epoch 5/200
45/45 [==============================] - 0s 10ms/step - loss: 0.1350 - accuracy: 0.9500 - val_loss: 0.5372 - val_accuracy: 0.7250 - lr: 0.0600
Epoch 6/200
45/45 [==============================] - 0s 11ms/step - loss: 0.1106 - accuracy: 0.9667 - val_loss: 0.4254 - val_accuracy: 0.8250 - lr: 0.0360
Epoch 7/200
45/45 [==============================] - 1s 11ms/step - loss: 0.1174 -

...

45/45 [==============================] - 2s 28ms/step - loss: 1.7096 - accuracy: 0.7389 - val_loss: 0.5808 - val_accuracy: 0.6500 - lr: 0.1000
Epoch 2/200
45/45 [==============================] - 3s 69ms/step - loss: 0.3666 - accuracy: 0.8528 - val_loss: 0.6177 - val_accuracy: 0.6500 - lr: 0.1000
Epoch 3/200
45/45 [==============================] - 2s 41ms/step - loss: 0.2756 - accuracy: 0.8750 - val_loss: 0.6169 - val_accuracy: 0.6500 - lr: 0.0600
Epoch 4/200
45/45 [==============================] - 2s 45ms/step - loss: 0.2079 - accuracy: 0.9167 - val_loss: 0.6122 - val_accuracy: 0.6500 - lr: 0.0600
Epoch 5/200
45/45 [==============================] - 2s 47ms/step - loss: 0.1696 - accuracy: 0.9333 - val_loss: 0.5174 - val_accuracy: 0.6750 - lr: 0.0600
Epoch 6/200
45/45 [==============================] - 2s 50ms/step - loss: 0.1457 - accuracy: 0.9389 - val_loss: 0.5534 - val_accuracy: 0.6500 - lr: 0.0360
Epoch 7/200
45/45 [==============================] - 3s 57ms/step - loss: 0.1217 -

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


45/45 [==============================] - 3s 45ms/step - loss: 1.6087 - accuracy: 0.7361 - val_loss: 0.7174 - val_accuracy: 0.5000 - lr: 0.1000
Epoch 2/200
45/45 [==============================] - 2s 44ms/step - loss: 0.2945 - accuracy: 0.8639 - val_loss: 0.6782 - val_accuracy: 0.5000 - lr: 0.1000
Epoch 3/200
45/45 [==============================] - 3s 66ms/step - loss: 0.2359 - accuracy: 0.9222 - val_loss: 0.6785 - val_accuracy: 0.5000 - lr: 0.0600
Epoch 4/200
45/45 [==============================] - 2s 42ms/step - loss: 0.1558 - accuracy: 0.9500 - val_loss: 0.6661 - val_accuracy: 0.5000 - lr: 0.0600
Epoch 5/200
45/45 [==============================] - 2s 48ms/step - loss: 0.1461 - accuracy: 0.9528 - val_loss: 0.6029 - val_accuracy: 0.6000 - lr: 0.0600
Epoch 6/200
45/45 [==============================] - 2s 35ms/step - loss: 0.1071 - accuracy: 0.9750 - val_loss: 0.5196 - val_accuracy: 0.7250 - lr: 0.0360
Epoch 7/200
45/45 [==============================] - 2s 37ms/step - loss: 0.0883 -

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


45/45 [==============================] - 2s 26ms/step - loss: 1.1578 - accuracy: 0.6917 - val_loss: 0.8425 - val_accuracy: 0.4000 - lr: 0.1000
Epoch 2/200
45/45 [==============================] - 1s 32ms/step - loss: 0.5177 - accuracy: 0.7972 - val_loss: 0.9528 - val_accuracy: 0.4000 - lr: 0.1000
Epoch 3/200
45/45 [==============================] - 1s 26ms/step - loss: 0.4147 - accuracy: 0.8389 - val_loss: 0.9646 - val_accuracy: 0.4000 - lr: 0.0600
Epoch 4/200
45/45 [==============================] - 2s 41ms/step - loss: 0.3482 - accuracy: 0.8444 - val_loss: 0.8721 - val_accuracy: 0.4000 - lr: 0.0600
Epoch 5/200
45/45 [==============================] - 2s 36ms/step - loss: 0.2860 - accuracy: 0.8833 - val_loss: 0.8633 - val_accuracy: 0.4000 - lr: 0.0600
Epoch 6/200
45/45 [==============================] - 1s 29ms/step - loss: 0.2260 - accuracy: 0.9083 - val_loss: 0.8242 - val_accuracy: 0.4000 - lr: 0.0360
Epoch 7/200
45/45 [==============================] - 1s 34ms/step - loss: 0.2112 -

In [ ]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.9375 ± 0.027003086243366056
0.9422132397342217 ± 0.027986222699787806
0.9152546199033613 ± 0.06341920671767641
0.969171859565082 ± 0.03911949357765075
0.877870358964356 ± 0.04983169331996202
0.9682335208987181 ± 0.024956663570442208


### model evaluation in test dataset

In [ ]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


50/50 [==============================] - 2s 30ms/step - loss: 1.5917 - accuracy: 0.7800 - val_loss: 0.6248 - val_accuracy: 0.7500 - lr: 0.1000
Epoch 2/200
50/50 [==============================] - 1s 21ms/step - loss: 0.3273 - accuracy: 0.8825 - val_loss: 0.5830 - val_accuracy: 0.7750 - lr: 0.1000
Epoch 3/200
50/50 [==============================] - 2s 36ms/step - loss: 0.2220 - accuracy: 0.9150 - val_loss: 0.5402 - val_accuracy: 0.7750 - lr: 0.0600
Epoch 4/200
50/50 [==============================] - 2s 30ms/step - loss: 0.1960 - accuracy: 0.9250 - val_loss: 0.4951 - val_accuracy: 0.9500 - lr: 0.0600
Epoch 5/200
50/50 [==============================] - 2s 41ms/step - loss: 0.1404 - accuracy: 0.9500 - val_loss: 0.4506 - val_accuracy: 0.8500 - lr: 0.0600
Epoch 6/200
50/50 [==============================] - 2s 40ms/step - loss: 0.1197 - accuracy: 0.9550 - val_loss: 0.3049 - val_accuracy: 0.9000 - lr: 0.0360
Epoch 7/200
50/50 [==============================] - 2s 33ms/step - loss: 0.0998 -

In [ ]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.95
0.9545454545454546
0.9090909090909091
1.0
0.9045340337332909
0.99


In [ ]:
model.save('QS_tensorflow_model',save_format = 'tf') 
!zip -r /content/QS_tensorflow_model.zip /content/QS_tensorflow_model

  adding: content/QS_tensorflow_model/ (stored 0%)
  adding: content/QS_tensorflow_model/variables/ (stored 0%)
  adding: content/QS_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/QS_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 40%)
  adding: content/QS_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/QS_tensorflow_model/assets/ (stored 0%)
  adding: content/QS_tensorflow_model/keras_metadata.pb (deflated 89%)
